In [1]:
from langchain.prompts import PromptTemplate

In [2]:
# from dotenv import load_dotenv

# load_dotenv()  # take environment variables from .env.

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Question(BaseModel):
    """ A question about a domain of study."""

    topic_description: str = Field(
        description="A sentence describing the sub-topic to which the question belongs. That means this sentence should specify in a granular level what specific sub-topic the question belongs to. It should be abstract in a way that other questions could be put in this description too. Use between 5 and 10 words."
    )
    level: float = Field(
        description="The difficulty level of the question. It should be only one of the following options: 'easy', 'medium', 'hard'."
    )
    question: str = Field(
        description="The actual question text. It should be a question of type TRUE or FALSE. It means that the questions should be an assertion that could be answered with TRUE or FALSE."
    )
    answer_correct: str = Field(
        description="The correct answer to the question. It should be only one of the following options: TRUE or FALSE"
    )
    explanation: str = Field(
        description="An explanation or solution to the question."
    )

class QuestionList(BaseModel):
    """A list of Question class."""

    items: list[Question] = Field(
        description="list of Question"
    )

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_google = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_google_QuestionList = llm_google.with_structured_output(QuestionList)

/workspaces/llm-quiz-creator-streamlitapp-trainer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1723753125.961714    3301 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.


In [5]:
prompt_question_generator = PromptTemplate.from_template("""

    TASK CONTEXT:
    I am studying machine learning and I need to practice some questions on various topics.
    
    TASK DESCRIPTION:
    I will provide you with a list of topics, and I would like you to generate a list of TRUE or FALSE questions.
    These questions should be interesting, creative, challenging and thought-provoking. 
    Each question should be in the form of a statement that could be either TRUE or FALSE.
    Feel free to be imaginative and attempt to confuse the student by blending related concepts or similar words.
    I will provide the topics in the DOMAIN KNOWLEDGE section.
    The questions should pertain to these topics, and you can use this knowledge as a foundation to create questions that delve deeper into the subject matter.
    
    ADDITIONAL TASK DESCRIPTION:
    {additional_task_description}
    
    TASK REQUIREMENTS:
    Please refrain from creating questions that require mathematical calculations, but you may create questions with mathematical formulas.
    You SHOULD use LATEX to write mathematical formulas and code, but you should use the Katex flavor.
    Also you should put $$ in the beggining of the katex code and $$ at the end of the code. This is necessary because the interpreter needs it.
    
    TASK DETAILS:
    You should create {quantity} questions of level {level}.
    
    DOMAIN KNOWLEDGE:
    {domain_knowledge}
    
    FORMAT OUTPUT INSTRUCTIONS:
    It should be formatted as described in the output format.
    """
)

In [6]:
# from langchain_core.messages import AIMessage, HumanMessage
# from typing import List, Optional
# import  json_repair

# def json_parser(message: AIMessage) -> List[dict]:
#     return json_repair.loads(message.content)

In [7]:
text = """

Language Models and Probability
Entropy
Cross-Entropy
Cross-Entropy Loss
Perplexity

"""

In [8]:
def generate_questions(text, llm, parameters, subject_matter_1, subject_matter_2):
    print( "------------------- generate_questions FUNCTION -------------------" )
    
    if text is None:
        raise Exception("text is None")
        
    try:
        chain = prompt_question_generator | llm
        
        parameters["domain_knowledge"] = text
        
        questions = chain.invoke(parameters)
        
        # questions = json_parser(response)
        
        for q in questions:
            q["subject_matter_1"] = subject_matter_1
            # q["subject_matter_2"] = subject_matter_2    
        
            print("-----------------------questions-----------------------")
            print(q)
            

        # data, count = supabase.table('questions').insert(questions).execute()
        
    except Exception as e:
        print("An error occurred:", e)

In [9]:
for level in [
    # "beginner", 
    # "intermediate", 
    "hard"
    ]:
    print( "level:", level )
    
    parameters = {
        "quantity": 1,
        "level": level,
        "additional_task_description": "Create questions only about the definitions of the concepts, like mixing the definition of one with another distribution, or mixing the use of one with the use of another. I need this to memorize this concepts."
    }
    
    subject_matter_1 = "Probability - LLMs"
    subject_matter_2 = subject_matter_1
    
    generate_questions(text, llm_google_QuestionList, parameters, subject_matter_1, subject_matter_2)

level: hard
------------------- generate_questions FUNCTION -------------------
An error occurred: Invalid argument provided to Gemini: 400 * GenerateContentRequest.tools[0].function_declarations[0].parameters.properties[items].items: missing field.

